# FreqAI Colab Remote Execution Setup

This notebook sets up SSH server with ngrok tunnel for remote backtest execution.

## Steps:
1. Enable GPU (Runtime > Change runtime type > GPU)
2. Run all cells
3. Copy tunnel URL and password
4. Use from local machine with backtest_executor.py

In [ ]:
# Check GPU
import subprocess
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print('✅ GPU Available!')
    for line in result.stdout.split('\n'):
        if 'Tesla' in line or 'T4' in line or 'MiB' in line:
            print(line)
except:
    print('❌ GPU not found!')
    print('Enable GPU: Runtime > Change runtime type > GPU (T4)')

In [ ]:
%%capture
# Install SSH server
!apt-get update -qq
!apt-get install -y openssh-server pwgen

In [ ]:
# Configure and start SSH
import secrets
import string

# Generate random password
password = ''.join(secrets.choice(string.ascii_letters + string.digits) for _ in range(16))
print(f'🔑 SSH Password: {password}')

# Configure SSH
!mkdir -p /var/run/sshd
!echo "root:{password}" | chpasswd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!echo "Port 22" >> /etc/ssh/sshd_config

# Start SSH service
!service ssh restart

print('✅ SSH server started on port 22')

In [ ]:
# Download and setup ngrok
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz
!chmod +x ngrok

print('✅ ngrok downloaded')
print('\n📝 Get your ngrok token from: https://dashboard.ngrok.com/get-started/your-authtoken')

In [ ]:
# Start ngrok tunnel
import subprocess
import time
import requests
import getpass

# Get ngrok token from user
ngrok_token = getpass.getpass('Enter your ngrok auth token: ')

# Authenticate ngrok
!./ngrok authtoken {ngrok_token}

# Start ngrok tunnel in background
ngrok_process = subprocess.Popen(
    ['./ngrok', 'tcp', '22'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print('⏳ Starting tunnel...')
time.sleep(5)

# Get tunnel URL
try:
    response = requests.get('http://localhost:4040/api/tunnels')
    tunnels = response.json()
    if tunnels.get('tunnels'):
        tunnel_url = tunnels['tunnels'][0]['public_url']
        print(f'\n✅ Tunnel established!')
        print(f'\n{'='*70}')
        print('CONNECTION DETAILS')
        print('='*70)
        print(f'Tunnel URL: {tunnel_url}')
        print(f'Password: {password}')
        print('='*70)
        print(f'\n📋 Use these on local machine:')
        print(f'   --tunnel-url {tunnel_url}')
        print(f'   --password {password}')
    else:
        print('❌ No tunnel found. Check ngrok logs.')
except Exception as e:
    print(f'❌ Error: {e}')
    print('Make sure ngrok token is valid')

In [ ]:
%%capture install_output
import os

# Clone repository
print('📥 Cloning repository...')
!rm -rf freqai-futures-strategy
!git clone https://github.com/aminak58/freqai-futures-strategy.git
os.chdir('/content/freqai-futures-strategy')

# Install dependencies
print('📦 Installing dependencies...')
!pip install -q ta-lib-binary
!pip install -q 'freqtrade[freqai]'

print('✅ Setup complete!')

In [ ]:
# Show installation result
import os
print('✅ Setup complete!')
print(f'📂 Working directory: {os.getcwd()}')
print(f'\n📁 Files:')
!ls -la

In [ ]:
# Optional: Mount Google Drive for data
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Check for data
drive_zip = '/content/drive/MyDrive/FreqAI/binance_data.zip'
if os.path.exists(drive_zip):
    print('📦 Extracting data from Drive...')
    os.makedirs('/content/freqai-futures-strategy/user_data/data', exist_ok=True)
    with zipfile.ZipFile(drive_zip, 'r') as zip_ref:
        zip_ref.extractall('/content/freqai-futures-strategy/user_data/data/')
    print('✅ Data extracted!')
else:
    print('⚠️ No data found in Drive')
    print(f'Expected location: {drive_zip}')

In [ ]:
# Keep session alive
import time
from IPython.display import clear_output

print('🔄 Keeping session alive...')
print('='*70)
print('CONNECTION DETAILS (save these!)')
print('='*70)
print(f'Tunnel URL: {tunnel_url}')
print(f'Password: {password}')
print('='*70)
print('\n⏳ Session will auto-refresh every 5 minutes')
print('Press Stop button to end session\n')

try:
    counter = 0
    while True:
        time.sleep(300)  # 5 minutes
        counter += 1
        clear_output(wait=True)
        print(f'🔄 Session alive (refresh #{counter})')
        print('='*70)
        print('CONNECTION DETAILS')
        print('='*70)
        print(f'Tunnel URL: {tunnel_url}')
        print(f'Password: {password}')
        print('='*70)
        print(f'⏰ Time: {time.strftime("%H:%M:%S")}')
        print(f'🔢 Refreshed {counter} times')
        print('\nPress Stop button to end session')
except KeyboardInterrupt:
    print('\n✋ Session monitoring stopped')